# Request data from Yoga-82 dataset and place the .txt files in this folder

In [22]:
import os

from PIL import Image
import requests
import urllib.request as urllib2
import shutil
from bs4 import BeautifulSoup
from io import BytesIO
import pandas as pd
import numpy as np
import os.path as path
import time


In [23]:
# GLOBALS

OUTPUT_FOLDER = "output"
OUTPUT_CSV = "\output.csv"

In [24]:
main_dataset = pd.read_csv("dataset.csv", error_bad_lines=False, decimal=",", header=0,
                 usecols=
                     [
                         "format","id","url", "authors", "categories", "description", "isbn10", "isbn13", "lang", "publication-date", "title"
                     ]
                , dtype={'id':str}
                )

filter = (main_dataset["format"]=="9") | (main_dataset["format"]=="1") | (main_dataset["format"]=="2")
main_dataset.where(filter, inplace = True)
main_dataset = main_dataset.dropna()
main_dataset = main_dataset.filter(["format", "id", "url"])

In [25]:
if(path.exists(OUTPUT_FOLDER + OUTPUT_CSV)):
    output_dataframe = pd.read_csv(OUTPUT_FOLDER + OUTPUT_CSV, error_bad_lines=False, decimal=",", header=0, dtype={'id':str})
    print("Previous output detected: Loading")
else:
    output_dataframe = pd.DataFrame(columns = ['id', 'url','img_url'])
    print("No previous output detected: started with empty dataframe instead")

Previous output detected: Loading


In [29]:
#output_dataframe
main_dataset

,format,id,url
0,1,9780141036144,/1984-George-Orwell/9780141036144
1,1,9781501143908,/100-Deadly-Skills-Survival-Edition-Clint-Emer...
2,1,9781444762051,/100-Most-Pointless-Things-World-Alexander-Arm...
3,1,9781784943462,/50-Knitted-Dolls-Sarah-Keen/9781784943462
4,1,9781782216957,/20-Knit-Pocket-Pets-Sachiyo-Ishii/9781782216957
...,...,...,...
56003,1,9780711231207,/100-Greatest-Cycling-Climbs-Simon-Warren/9780...
56004,1,9780648696704,/2020-Biddy-Tarot-Planner-Brigit-Esselmont/978...
56005,1,9781472143051,/100-Greatest-Cycling-Climbs-Italy-Simon-Warre...
56006,1,9780692792889,/40-Winks-Brenda-Moore/9780692792889


In [27]:
def loop_through_df(main_dataset ,output_dataframe):
    bookId = ""
    url = ""
    indexSinceError = 0
    
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print('Created output folder as none was present')
    
    for index in range(len(main_dataset)):
        print(index)
        
        if(index < 50000): #Set limit, unless you want it to run 24/7, which I wouldn't.
            bookId = main_dataset.iloc[index,1]
            url = "https://www.bookdepository.com" + str(main_dataset.iloc[index,2])

            if(bookId in output_dataframe.id.values):
                print(bookId + " found in output_dataframe, skipping")
                continue

            response = requests.get(url)

            if(response.status_code != 200):
                if(response.status_code == 404):
                    print('Book not found, skipping')
                    continue
                if(index - indexSinceError > 50): #Amount of successfull attempts required until it errors again.
                    print("Blocked/Errored, waiting 5 minutes before retry")
                    time.sleep(300)
                    response = requests.get(url)
                    
                    if(response.status_code != 200):
                        print("Retry failed, stopping.")
                        break
        
                    indexSinceError = index;
                else:
                    print("Blocked too shortly after unblocked, stopping.")
                    break

            parsed_html = BeautifulSoup(response.text)
            img_elem = parsed_html.findAll("img", {"class": "book-img"})
            img_url = img_elem[0]['src']
            output_dataframe = output_dataframe.append(pd.DataFrame(data = [[bookId, url, img_url]], columns = ['id', 'url','img_url']), ignore_index = True)
            
        else: #Break after limit is reached.
            print("Limit reached, stopping")
            break
            
#             unused code for getting image, keeping it here for use later.
#             img = Image.open(BytesIO(response.content))
#             print(os.path.basename(name))
#             response = img.save(path + os.path.basename(name))
#             continue

    return output_dataframe;

In [28]:
result = loop_through_df(main_dataset ,output_dataframe)
print(result)
result.to_csv(OUTPUT_FOLDER + OUTPUT_CSV, index=False)
print("Saved dataframe");

0
9780141036144 found in output_dataframe, skipping
1
9781501143908 found in output_dataframe, skipping
2
9781444762051 found in output_dataframe, skipping
3
9781784943462 found in output_dataframe, skipping
4
9781782216957 found in output_dataframe, skipping
5
9781557830999 found in output_dataframe, skipping
6
9780809057399 found in output_dataframe, skipping
7
9780007123742 found in output_dataframe, skipping
8
9780306812989 found in output_dataframe, skipping
9
9780486225500 found in output_dataframe, skipping
10
9782912925534 found in output_dataframe, skipping
11
9780380798049 found in output_dataframe, skipping
12
9780345459145 found in output_dataframe, skipping
13
9781624142017 found in output_dataframe, skipping
14
9781542963305 found in output_dataframe, skipping
15
9780802130129 found in output_dataframe, skipping
16
9780750646963 found in output_dataframe, skipping
17
9780071443258 found in output_dataframe, skipping
18
9781460265048 found in output_dataframe, skipping
19


9780826486332 found in output_dataframe, skipping
195
9781603293846 found in output_dataframe, skipping
196
9780199560059 found in output_dataframe, skipping
197
9781847061522 found in output_dataframe, skipping
198
9780198854845 found in output_dataframe, skipping
199
9780578221397 found in output_dataframe, skipping
200
9781138336766 found in output_dataframe, skipping
201
9781925921625 found in output_dataframe, skipping
202
9783110440881 found in output_dataframe, skipping
203
9781712351703 found in output_dataframe, skipping
204
9780253049971 found in output_dataframe, skipping
205
9781786828859 found in output_dataframe, skipping
206
9780916260439 found in output_dataframe, skipping
207
9780817355548 found in output_dataframe, skipping
208
9781692891558 found in output_dataframe, skipping
209
9781692867645 found in output_dataframe, skipping
210
9780313224942 found in output_dataframe, skipping
211
9781978324039 found in output_dataframe, skipping
212
9781978293045 found in outpu

9781693032875 found in output_dataframe, skipping
407
9781693032950 found in output_dataframe, skipping
408
9781566082006 found in output_dataframe, skipping
409
9781416628583 found in output_dataframe, skipping
410
9780274141883 found in output_dataframe, skipping
411
9780615900476 found in output_dataframe, skipping
412
9781711551203 found in output_dataframe, skipping
413
9789400983304 found in output_dataframe, skipping
414
9781913063818 found in output_dataframe, skipping
415
9783642236556 found in output_dataframe, skipping
416
9781913063733 found in output_dataframe, skipping
417
9780863886782 found in output_dataframe, skipping
418
9780140481396 found in output_dataframe, skipping
419
9788478273379 found in output_dataframe, skipping
420
9781377574271 found in output_dataframe, skipping
421
9780270214352 found in output_dataframe, skipping
422
9780270343298 found in output_dataframe, skipping
423
9780270408607 found in output_dataframe, skipping
424
9780270188646 found in outpu

9781462526093 found in output_dataframe, skipping
618
9788498414264 found in output_dataframe, skipping
619
9781788739115 found in output_dataframe, skipping
620
9780007427376 found in output_dataframe, skipping
621
9788416498246 found in output_dataframe, skipping
622
9781641816045 found in output_dataframe, skipping
623
9781481452465 found in output_dataframe, skipping
624
9780528017391 found in output_dataframe, skipping
625
9781581960099 found in output_dataframe, skipping
626
9783844329902 found in output_dataframe, skipping
627
9783844327816 found in output_dataframe, skipping
628
9781477847893 found in output_dataframe, skipping
629
9781563472091 found in output_dataframe, skipping
630
9781328995056 found in output_dataframe, skipping
631
9780199672240 found in output_dataframe, skipping
632
9781977593993 found in output_dataframe, skipping
633
9780971534476 found in output_dataframe, skipping
634
9780310730477 found in output_dataframe, skipping
635
9781849920568 found in outpu

9781165393725 found in output_dataframe, skipping
830
9781165282593 found in output_dataframe, skipping
831
9781167211270 found in output_dataframe, skipping
832
9781167252099 found in output_dataframe, skipping
833
9781167251887 found in output_dataframe, skipping
834
9781167230011 found in output_dataframe, skipping
835
9781931282314 found in output_dataframe, skipping
836
9780684856094 found in output_dataframe, skipping
837
9781305639034 found in output_dataframe, skipping
838
9780810126404 found in output_dataframe, skipping
839
9780198777250 found in output_dataframe, skipping
840
9781464401565 found in output_dataframe, skipping
841
9780367243906 found in output_dataframe, skipping
842
9780687090846 found in output_dataframe, skipping
843
9781703592627 found in output_dataframe, skipping
844
9780071377591 found in output_dataframe, skipping
845
9786137770443 found in output_dataframe, skipping
846
9788365853530 found in output_dataframe, skipping
847
9786138064404 found in outpu

...

9788416012046 found in output_dataframe, skipping
44553
9781482439632 found in output_dataframe, skipping
44554
9781981255603 found in output_dataframe, skipping
44555
9781723841392 found in output_dataframe, skipping
44556
9781979614641 found in output_dataframe, skipping
44557
9781600780240 found in output_dataframe, skipping
44558
9781478394334 found in output_dataframe, skipping
44559
9781475125726 found in output_dataframe, skipping
44560
9781797482712 found in output_dataframe, skipping
44561
9781791796174 found in output_dataframe, skipping
44562
9780786477456 found in output_dataframe, skipping
44563
9780786476558 found in output_dataframe, skipping
44564
9780811737852 found in output_dataframe, skipping
44565
9780992032708 found in output_dataframe, skipping
44566
9781412003254 found in output_dataframe, skipping
44567
9780786440658 found in output_dataframe, skipping
44568
9781921517808 found in output_dataframe, skipping
44569
9781681570570 found in output_dataframe, skippin

9781507749548 found in output_dataframe, skipping
44793
9788370207335 found in output_dataframe, skipping
44794
9781503024342 found in output_dataframe, skipping
44795
9781503235977 found in output_dataframe, skipping
44796
9780593074596 found in output_dataframe, skipping
44797
9781635313161 found in output_dataframe, skipping
44798
9781514228852 found in output_dataframe, skipping
44799
9781847717405 found in output_dataframe, skipping
44800
9781084154360 found in output_dataframe, skipping
44801
9781085800273 found in output_dataframe, skipping
44802
9781517739966 found in output_dataframe, skipping
44803
9781519369031 found in output_dataframe, skipping
44804
9781519526779 found in output_dataframe, skipping
44805
9781519526595 found in output_dataframe, skipping
44806
9781519451149 found in output_dataframe, skipping
44807
9781095110034 found in output_dataframe, skipping
44808
9781793806727 found in output_dataframe, skipping
44809
9781523207763 found in output_dataframe, skippin

9780312308858 found in output_dataframe, skipping
45043
9781941525746 found in output_dataframe, skipping
45044
9781890946463 found in output_dataframe, skipping
45045
9783743113343 found in output_dataframe, skipping
45046
9781847971869 found in output_dataframe, skipping
45047
9780786410477 found in output_dataframe, skipping
45048
9781168831576 found in output_dataframe, skipping
45049
9781168849786 found in output_dataframe, skipping
45050
9781165981793 found in output_dataframe, skipping
45051
9781166047726 found in output_dataframe, skipping
45052
9781166046361 found in output_dataframe, skipping
45053
9781166033125 found in output_dataframe, skipping
45054
9781097232147 found in output_dataframe, skipping
45055
9781097443277 found in output_dataframe, skipping
45056
9781166030469 found in output_dataframe, skipping
45057
9781517491666 found in output_dataframe, skipping
45058
9781517508685 found in output_dataframe, skipping
45059
9781517740818 found in output_dataframe, skippin

9780786403783 found in output_dataframe, skipping
45287
9781660773671 found in output_dataframe, skipping
45288
9781660067282 found in output_dataframe, skipping
45289
9781167213588 found in output_dataframe, skipping
45290
9781167115813 found in output_dataframe, skipping
45291
9780767931908 found in output_dataframe, skipping
45292
9780873389860 found in output_dataframe, skipping
45293
9781426973680 found in output_dataframe, skipping
45294
9781425974442 found in output_dataframe, skipping
45295
9781541064560 found in output_dataframe, skipping
45296
9781544044064 found in output_dataframe, skipping
45297
9781493033737 found in output_dataframe, skipping
45298
9781859608456 found in output_dataframe, skipping
45299
9780999183625 found in output_dataframe, skipping
45300
9781541197008 found in output_dataframe, skipping
45301
9781495265037 found in output_dataframe, skipping
45302
9781782550532 found in output_dataframe, skipping
45303
9781138085374 found in output_dataframe, skippin

9781646721948 found in output_dataframe, skipping
45531
9781646721931 found in output_dataframe, skipping
45532
9781646721573 found in output_dataframe, skipping
45533
9781646721870 found in output_dataframe, skipping
45534
9781646721849 found in output_dataframe, skipping
45535
9781646721900 found in output_dataframe, skipping
45536
9781646721726 found in output_dataframe, skipping
45537
9781646721443 found in output_dataframe, skipping
45538
9781646720675 found in output_dataframe, skipping
45539
9781646720651 found in output_dataframe, skipping
45540
9781646720682 found in output_dataframe, skipping
45541
9781646720415 found in output_dataframe, skipping
45542
9781646720125 found in output_dataframe, skipping
45543
9781646720514 found in output_dataframe, skipping
45544
9781646720491 found in output_dataframe, skipping
45545
9781646720507 found in output_dataframe, skipping
45546
9781646720323 found in output_dataframe, skipping
45547
9781646720262 found in output_dataframe, skippin

In [ ]:
# img_url_dataframe = pd.DataFrame(columns = ['id', 'url','img_url'])

# img_url_dataframe

# img_url_dataframe = img_url_dataframe.append(pd.DataFrame(data = 
#                                                           [['9780141036144',
#                                                             'https://www.bookdepository.com/1984-George-Orwell/9780141036144',
#                                                             'https://d1w7fb2mkkr3kw.cloudfront.net/assets/images/book/lrg/9780/1410/9780141036144.jpg']], columns = ['id', 'url','img_url']))

# img_url_dataframe



# response = requests.get("https://www.bookdepository.com/1984-George-Orwell/9780141036144")
# parsed_html = BeautifulSoup(response.text)
# img_elem = parsed_html.findAll("img", {"class": "book-img"})
# img_url = img_elem[0]['src']
# print(img_url)
# img_url_dataframe = img_url_dataframe.append(pd.DataFrame(data = ['9780141036144', img_url], columns = ['id', 'url','img_url']))


